In [19]:
import requests   # best library to manage HTTP transactions
from bs4 import BeautifulSoup # web-scraping library

acceptMediaType = 'text/html'
userAgentHeader = 'BaskaufScraper/0.1 (steve.baskauf@vanderbilt.edu)'
requestHeaderDictionary = {
    'Accept' : acceptMediaType,
    'User-Agent': userAgentHeader
}
url = 'https://as.vanderbilt.edu/biosci/people/index.php?group=secondary-faculty'
response = requests.get(url, headers = requestHeaderDictionary)
soupObject = BeautifulSoup(response.text,features="html5lib")

In [20]:
tableObject = soupObject.find_all('table')[0]
print(tableObject)

<table class="table table-striped" width="100%"><tbody><tr><td width="100"><a href="/biosci/bio/suzana-herculano-houzel"><img class="img-rounded" src="/biosci/images/people/image.php/suzana-herculano-houzel.jpg?width=100&amp;image=/biosci/images/people/suzana-herculano-houzel.jpg"/></a></td><td align="left" class="biolink"><strong><a href="/biosci/bio/suzana-herculano-houzel">Suzana Herculano-Houzel</a></strong><br/>Associate Professor of Psychology<br/>Associate Professor of Biological Sciences</td><td class="contactinfo" width="25%"><a href="mailto:Suzana.herculano@vanderbilt.edu">Email</a><br/><a href="http://www.suzanaherculanohouzel.com/lab">Website</a><br/>615-875-9086<br/>066 Wilson Hall</td></tr>
<tr><td width="100"><a href="/biosci/bio/jake-hughey"><img class="img-rounded" src="/biosci/images/people/image.php/jake-hughey.jpg?width=100&amp;image=/biosci/images/people/jake-hughey.jpg"/></a></td><td align="left" class="biolink"><strong><a href="/biosci/bio/jake-hughey">Jake Hughe

In [21]:
rowObjectsList = tableObject.find_all('tr')
print(rowObjectsList)

[<tr><td width="100"><a href="/biosci/bio/suzana-herculano-houzel"><img class="img-rounded" src="/biosci/images/people/image.php/suzana-herculano-houzel.jpg?width=100&amp;image=/biosci/images/people/suzana-herculano-houzel.jpg"/></a></td><td align="left" class="biolink"><strong><a href="/biosci/bio/suzana-herculano-houzel">Suzana Herculano-Houzel</a></strong><br/>Associate Professor of Psychology<br/>Associate Professor of Biological Sciences</td><td class="contactinfo" width="25%"><a href="mailto:Suzana.herculano@vanderbilt.edu">Email</a><br/><a href="http://www.suzanaherculanohouzel.com/lab">Website</a><br/>615-875-9086<br/>066 Wilson Hall</td></tr>, <tr><td width="100"><a href="/biosci/bio/jake-hughey"><img class="img-rounded" src="/biosci/images/people/image.php/jake-hughey.jpg?width=100&amp;image=/biosci/images/people/jake-hughey.jpg"/></a></td><td align="left" class="biolink"><strong><a href="/biosci/bio/jake-hughey">Jake Hughey, Ph.D.</a></strong><br/>Assistant Professor of Biom

In [22]:
rowObject = rowObjectsList[0]
cellObjectsList = rowObject.find_all('td')
print(cellObjectsList)

[<td width="100"><a href="/biosci/bio/suzana-herculano-houzel"><img class="img-rounded" src="/biosci/images/people/image.php/suzana-herculano-houzel.jpg?width=100&amp;image=/biosci/images/people/suzana-herculano-houzel.jpg"/></a></td>, <td align="left" class="biolink"><strong><a href="/biosci/bio/suzana-herculano-houzel">Suzana Herculano-Houzel</a></strong><br/>Associate Professor of Psychology<br/>Associate Professor of Biological Sciences</td>, <td class="contactinfo" width="25%"><a href="mailto:Suzana.herculano@vanderbilt.edu">Email</a><br/><a href="http://www.suzanaherculanohouzel.com/lab">Website</a><br/>615-875-9086<br/>066 Wilson Hall</td>]


In [23]:
nameCell = cellObjectsList[1]
print(nameCell)
wholeText = nameCell.text
name = nameCell('a')[0].text
print(name)
titleText = wholeText.partition(name)[2]
print(titleText)

<td align="left" class="biolink"><strong><a href="/biosci/bio/suzana-herculano-houzel">Suzana Herculano-Houzel</a></strong><br/>Associate Professor of Psychology<br/>Associate Professor of Biological Sciences</td>
Suzana Herculano-Houzel
Associate Professor of PsychologyAssociate Professor of Biological Sciences


In [37]:
import requests   # best library to manage HTTP transactions
from bs4 import BeautifulSoup # web-scraping library
import csv

def writeCsv(fileName, array):
    fileObject = open(fileName, 'w', newline='', encoding='utf-8')
    writerObject = csv.writer(fileObject)
    for row in array:
        writerObject.writerow(row)
    fileObject.close()

acceptMediaType = 'text/html'
userAgentHeader = 'BaskaufScraper/0.1 (steve.baskauf@vanderbilt.edu)'
requestHeaderDictionary = {
    'Accept' : acceptMediaType,
    'User-Agent': userAgentHeader
}

outputTable = [['name', 'degree', 'role', 'category']]
categories = ['primary-training-faculty', 'research-and-teaching-faculty', 'secondary-faculty', 'postdoc-fellows', 'emeriti']
degreeList = [
    {'string': 'Ph.D.', 'value': 'Ph.D.'},
    {'string': 'PhD', 'value': 'Ph.D.'},
    {'string': 'D.Phil.', 'value': 'D.Phil.'},
    {'string': 'J.D.', 'value': 'J.D.'}
     ]

for category in categories:
    url = 'https://as.vanderbilt.edu/biosci/people/index.php?group=' + category
    response = requests.get(url, headers = requestHeaderDictionary)
    soupObject = BeautifulSoup(response.text,features="html5lib")

    # get the first table from the page
    tableObject = soupObject.find_all('table')[0]
    
    # get the rows from the table
    rowObjectsList = tableObject.find_all('tr')
    for rowObject in rowObjectsList:
        # get the cells from each row
        cellObjectsList = rowObject.find_all('td')
        # picture is in cell 0, name and title is in cell 1
        nameCell = cellObjectsList[1]
        wholeText = nameCell.text
        # the name part is bolded
        try:
            name = nameCell('strong')[0].text
            # the title part isn't in a tag, so subtract the name part from the whole text to get it
            titleText = wholeText.partition(name)[2]
            # separate degrees from names
            degree = ''
            for testDegree in degreeList:
                if testDegree['string'] in name:
                    name = name.partition(', ' + testDegree['string'])[0]
                    # correct any malformed strings
                    degree = testDegree['value']
        except:
            name = wholeText
            titleText = ''
            degree = ''
        outputTable.append([name, degree, titleText, category])            

fileName = 'bsci-employees.csv'
writeCsv(fileName, outputTable)
print('done')

done
